## CNN Based Autoencoder

Source: https://www.geeksforgeeks.org/implement-convolutional-autoencoder-in-pytorch-with-cuda/



In [13]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [20]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
from PIL import Image

# Define the autoencoder architecture
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16,
                               kernel_size=3,
                               stride=2,
                               padding=1,
                               output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3,
                               kernel_size=3,
                               stride=2,
                               padding=1,
                               output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Define PSNR calculation function
def calculate_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    max_pixel_value = 1.0  # Assuming images are normalized between 0 and 1

    psnr = 10 * np.log10((max_pixel_value ** 2) / mse)
    return psnr

# Define transform
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the absolute path to the "pneumonia" directory
pneumonia_dir = r'C:\Users\Dylan Sy\Desktop\biomed\ISCS_30.65\Submission #2\pneumonia'

# Train an autoencoder for each folder
for folder_name in ['25', '50', '75']:
    # Load dataset from 'noisy_train' and 'noisy_test' folders
    train_dataset = datasets.ImageFolder(root=os.path.join(pneumonia_dir, 'noisy_train', folder_name), transform=transform)
    test_dataset = datasets.ImageFolder(root=os.path.join(pneumonia_dir, 'noisy_test', folder_name), transform=transform)

    # Define the dataloader
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=128)

    # Initialize the autoencoder
    model = Autoencoder().to(device)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the autoencoder
    num_epochs = 50
    for epoch in range(num_epochs):
        for data in train_loader:
            img, _ = data
            img = img.to(device)
            optimizer.zero_grad()
            output = model(img)
            loss = criterion(output, img)
            loss.backward()
            optimizer.step()
        if epoch % 5 == 0:
            print(f'Folder {folder_name}, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Evaluate PSNR on the test dataset
    psnr_total = 0.0
    num_samples = 0

    for i, (img, paths) in enumerate(test_loader):
        img = img.to(device)
        output = model(img)

        # Iterate over batch images
        for i in range(img.size(0)):
            index = num_samples + i
            if index < len(test_dataset.samples):  # Check if the index is within the range
                original_img_path = os.path.join('C:', 'Users', 'Dylan Sy', 'Desktop', 'biomed', 'ISCS_30.65', 'Submission #2', 'pneumonia', 'noisy_test', os.path.basename(test_dataset.samples[index][0]))
                original_img = np.array(Image.open(original_img_path).convert('RGB')) / 255.0  # Normalize to [0, 1]
                original_img_tensor = torch.tensor(original_img).permute(2, 0, 1).unsqueeze(0).float().to(device)

                # Resize the output to match the original image size
                resized_output = torch.nn.functional.interpolate(output[i].unsqueeze(0), size=original_img_tensor.shape[-2:], mode='bilinear', align_corners=False)

            # Calculate PSNR
            psnr = calculate_psnr(original_img_tensor.cpu().numpy(), resized_output.detach().cpu().numpy())
            psnr_total += psnr

    average_psnr = psnr_total / num_samples
    print('Average PSNR on test dataset: {:.4f} dB'.format(average_psnr))

    # Save the model
    torch.save(model.state_dict(), f'conv_autoencoder_{folder_name}.pth')


FileNotFoundError: Couldn't find any class folder in C:\Users\Dylan Sy\Desktop\biomed\ISCS_30.65\Submission #2\pneumonia\noisy_train\25.

In [1]:
import os
import torch
import pandas as pd
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define transform
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the original training dataset
original_train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/pneumonia/original_train', transform=transform)

# Initialize the autoencoder
model = Autoencoder().to(device)
model.load_state_dict(torch.load('conv_autoencoder_75.pth'))  # Load the trained model
model.eval()

# Create lists to store z values and identifiers
z_values = []
identifiers = []

# Iterate through the original training dataset
for img, paths in original_train_dataset:
    img = img.unsqueeze(0).to(device)  # Add batch dimension and move to device
    with torch.no_grad():
        z = model.encode(img)  # Get the z value using the encode method
    z_values.append(z.squeeze().cpu().numpy())  # Convert to numpy array and append to z_values
    identifiers.append(os.path.basename(paths))  # Store the identifier (e.g., filename)

# Convert lists to DataFrame
df = pd.DataFrame(z_values, columns=[f'z_{i}' for i in range(z_values[0].shape[0])])
df['identifier'] = identifiers

# Save DataFrame to CSV
df.to_csv('z_values_original_train.csv', index=False)


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/Colab Notebooks/pneumonia/original_train'